In [ ]:
# Cell 1 – podstawowy setup

from pathlib import Path
import sys

from pathlib import Path
from collections import Counter

from siwz_mapper.io.pdf_loader import load_pdf
from siwz_mapper.preprocess.segmenter import Segmenter
from siwz_mapper.llm.gpt_client import GPTClient
from siwz_mapper.llm.classify_segments import classify_segments
from siwz_mapper.pipeline.variant_aggregator import VariantAggregator
# Jeśli źródła są w folderze src/, dodaj go do sys.path
PROJECT_ROOT = Path(".").resolve()
SRC_DIR = PROJECT_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("SRC_DIR in sys.path:", str(SRC_DIR) in sys.path)
# Cell 1 – basic setup (ścieżki + sys.path)

from pathlib import Path
import sys

# Projekt zakładam, że odpalasz z katalogu głównego repo (tam, gdzie jest 'src' i np. 'scripts')
PROJECT_ROOT = Path(".").resolve()
SRC_DIR = PROJECT_ROOT / "src"

if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("SRC_DIR in sys.path:", str(SRC_DIR) in sys.path)

# Jedyny plik PDF do testów:
PDF_PATH = Path(
    r"C:\Programowanie\SIWZ\data\examples\OPZ_wybrane_strony.pdf"
)

print("PDF_PATH:", PDF_PATH)
print("PDF exists:", PDF_PATH.exists())
# Cell 2 – helpery

def preview_text(text: str, max_len: int = 200) -> str:
    t = text.replace("\n", " ⏎ ")
    return t if len(t) <= max_len else t[:max_len] + "…"

def print_segments_brief(segments, max_items: int = 20):
    print(f"Total segments: {len(segments)}\n")
    for s in segments[:max_items]:
        print(f"id={s.segment_id} page={s.page} len={len(s.text)} "
              f"chars={s.start_char}-{s.end_char}")
        print(preview_text(s.text))
        print("-" * 80)
# Cell 5 – uruchomienie Segmenter na tym PDF

# Cell 3 – test PDFLoader na JEDNYM PDF

from siwz_mapper.io.pdf_loader import PDFLoader, load_pdf

def inspect_pdf_loader(pdf_path: Path, show_first_blocks: int = 30):
    blocks = load_pdf(pdf_path, extract_bboxes=True)
    print(f"PDF: {pdf_path}")
    print(f"Number of blocks: {len(blocks)}\n")

    for i, b in enumerate(blocks[:show_first_blocks]):
        print(f"[{i}] id={b.segment_id} page={b.page}")
        print(f"bbox={b.bbox}")
        print(f"chars={b.start_char}-{b.end_char} (len={len(b.text)})")
        print(preview_text(b.text))
        print("-" * 80)

    return blocks

blocks = inspect_pdf_loader(PDF_PATH, show_first_blocks=30)

from siwz_mapper.preprocess.segmenter import Segmenter

def run_segmenter_on_main_pdf(pdf_path: Path, max_preview: int = 40):
    blocks = load_pdf(pdf_path, extract_bboxes=True)
    segmenter = Segmenter()
    segments = segmenter.segment(blocks)

    print(f"PDF: {pdf_path}")
    print(f"Num blocks:   {len(blocks)}")
    print(f"Num segments: {len(segments)}\n")

    print_segments_brief(segments, max_items=max_preview)
    return blocks, segments

blocks_seg, segments_seg = run_segmenter_on_main_pdf(PDF_PATH, max_preview=40)
def find_longest_by_text(objs):
    """
    Zwraca (najdłuższy_obiekt, długość_tekstu).
    Przyjmuje, że obiekt ma atrybut .text
    """
    if not objs:
        return None, 0
    
    longest = max(objs, key=lambda o: len(getattr(o, "text", "")))
    return longest, len(longest.text)


# Najdłuższy blok
longest_block, longest_block_len = find_longest_by_text(blocks)
print("=== NAJDŁUŻSZY BLOK ===")
print(f"Długość tekstu: {longest_block_len}")
# jeśli masz start_char / end_char / page, możesz też to pokazać:
if hasattr(longest_block, "page"):
    print(f"Strona: {longest_block.page}")
if hasattr(longest_block, "start_char") and hasattr(longest_block, "end_char"):
    print(f"Zakres znaków: {longest_block.start_char}–{longest_block.end_char}")
print("\nTreść bloku:\n")
print(longest_block.text)
print("\n" + "="*80 + "\n")

# Najdłuższy segment
longest_segment, longest_segment_len = find_longest_by_text(segments_seg)
print("=== NAJDŁUŻSZY SEGMENT ===")
print(f"Długość tekstu: {longest_segment_len}")
if hasattr(longest_segment, "page"):
    print(f"Strona: {getattr(longest_segment, 'page', 'N/A')}")
if hasattr(longest_segment, "start_char") and hasattr(longest_segment, "end_char"):
    print(f"Zakres znaków: {longest_segment.start_char}–{longest_segment.end_char}")
print("\nTreść segmentu:\n")
print(longest_segment.text)
import os

# 🔴 Najprościej (ale NIE wrzucaj tego do repo!)
os.environ["OPENAI_API_KEY"] = ""

# Kontrola:
print("OPENAI_API_KEY set:", bool(os.getenv("OPENAI_API_KEY")))
# Cell 1 – importy i pomocnicza funkcja podglądu


def preview_text(text: str, max_len: int = 160) -> str:
    """Prosty skrót tekstu do logów."""
    if not text:
        return ""
    t = " ".join(str(text).split())
    return t if len(t) <= max_len else t[:max_len] + " [...]"
# Cell 2 – główna funkcja: PDF -> GPT -> VariantAggregator

def aggregate_variants_on_pdf_with_gpt(
    pdf_path: Path,
    model_name: str = "gpt-4o-mini",
    max_segments: int | None = None,
    show_progress: bool = True,
):
    """
    Pełny mini-flow:
    PDF -> PDFLoader -> Segmenter -> GPTClient -> VariantAggregator

    Zapisuje wyniki do globalnych zmiennych:
      last_blocks
      last_segments
      last_classifications
      last_updated_segments
      last_variant_groups

    Dzięki temu możesz później analizować wyniki bez ponownego wołania GPT.
    """
    global last_blocks, last_segments, last_classifications
    global last_updated_segments, last_variant_groups

    pdf_path = Path(pdf_path)
    print(f"=== Running pipeline on: {pdf_path} ===")

    # 1) Bloki + segmenty
    blocks = load_pdf(pdf_path, extract_bboxes=True)
    segmenter = Segmenter()
    segments = segmenter.segment(blocks)

    if max_segments is not None:
        segments_to_use = segments[:max_segments]
    else:
        segments_to_use = segments

    print(f"Num blocks   : {len(blocks)}")
    print(f"Num segments : {len(segments)}")
    print(f"Using        : {len(segments_to_use)} segment(s) for GPT\n")

    # 2) Klient GPT
    client = GPTClient(model=model_name, temperature=0.0)

    # 3) Klasyfikacja
    classifications = classify_segments(
        segments_to_use,
        client,
        show_progress=show_progress,
    )

    # 4) Agregacja wariantów
    aggregator = VariantAggregator()
    updated_segments, variant_groups = aggregator.aggregate(
        segments_to_use,
        classifications,
    )

    print(f"\nnum_segments_used = {len(segments_to_use)}")
    print(f"num_variants      = {len(variant_groups)}\n")

    for vg in variant_groups:
        pages = []
        if vg.header_segment:
            pages.append(vg.header_segment.page)
        pages.extend([s.page for s in vg.body_segments])
        pages.extend([s.page for s in vg.prophylaxis_segments])

        if pages:
            span = f"{min(pages)}–{max(pages)}"
        else:
            span = "[no pages]"

        print(f"Variant {vg.variant_id}")
        print(f"  pages span   : {span}")
        print(f"  body         : {len(vg.body_segments)} segments")
        print(f"  prophylaxis  : {len(vg.prophylaxis_segments)} segments")
        if vg.header_segment:
            print("  header preview:",
                  preview_text(vg.header_segment.text, max_len=120))
        else:
            print("  header preview: [None]")
        print("-" * 80)

    # 5) zapis do globalnych zmiennych (do dalszej analizy)
    last_blocks = blocks
    last_segments = segments_to_use
    last_classifications = classifications
    last_updated_segments = updated_segments
    last_variant_groups = variant_groups

    return updated_segments, variant_groups
# Cell 3 – helper do szybkiego podsumowania wariantów

def summarize_last_variants():
    """
    Zwięzłe podsumowanie wariantów z ostatniego uruchomienia
    aggregate_variants_on_pdf_with_gpt.
    """
    try:
        variant_groups = last_variant_groups
    except NameError:
        print("Brak last_variant_groups – uruchom najpierw aggregate_variants_on_pdf_with_gpt().")
        return

    print("=== Summary of last_variant_groups ===\n")
    for vg in variant_groups:
        pages = []
        if vg.header_segment:
            pages.append(vg.header_segment.page)
        pages.extend([s.page for s in vg.body_segments])
        pages.extend([s.page for s in vg.prophylaxis_segments])

        span = f"{min(pages)}–{max(pages)}" if pages else "[no pages]"

        print(f"Variant {vg.variant_id}")
        print(f"  pages span     : {span}")
        print(f"  body segments  : {len(vg.body_segments)}")
        print(f"  prophylaxis seg: {len(vg.prophylaxis_segments)}")
        if vg.header_segment:
            print("  header preview:",
                  preview_text(vg.header_segment.text, max_len=160))
        print("-" * 70)


def show_last_variant_examples(variant_id: str, max_body: int = 5, max_proph: int = 5):
    """
    Pokaż przykładowe segmenty body/prophylaxis dla wybranego wariantu.
    """
    try:
        variant_groups = last_variant_groups
    except NameError:
        print("Brak last_variant_groups – uruchom najpierw aggregate_variants_on_pdf_with_gpt().")
        return

    vg = None
    for g in variant_groups:
        if g.variant_id == variant_id:
            vg = g
            break

    if vg is None:
        print(f"Nie znaleziono wariantu {variant_id}")
        return

    print(f"=== Examples for {vg.variant_id} ===")

    if vg.header_segment:
        print("\n--- HEADER ---")
        print(f"({vg.header_segment.segment_id}, page {vg.header_segment.page})")
        print(preview_text(vg.header_segment.text, max_len=200))
    else:
        print("\n[No header_segment]")

    print("\n--- BODY (first) ---")
    for s in vg.body_segments[:max_body]:
        print(f"({s.segment_id}, page {s.page})")
        print(preview_text(s.text, max_len=200))
        print("-" * 40)

    print("\n--- PROPHYLAXIS (first) ---")
    if vg.prophylaxis_segments:
        for s in vg.prophylaxis_segments[:max_proph]:
            print(f"({s.segment_id}, page {s.page})")
            print(preview_text(s.text, max_len=200))
            print("-" * 40)
    else:
        print("[No prophylaxis segments in this variant]")


PROJECT_ROOT: C:\Programowanie\SIWZ
SRC_DIR in sys.path: True
PROJECT_ROOT: C:\Programowanie\SIWZ
SRC_DIR in sys.path: True
PDF_PATH: C:\Programowanie\SIWZ\data\examples\OPZ_wybrane_strony.pdf
PDF exists: True
PDF: C:\Programowanie\SIWZ\data\examples\OPZ_wybrane_strony.pdf
Number of blocks: 119

[0] id=seg_p1_b0 page=1
bbox=page=1 x0=423.30999755859375 y0=73.23599243164062 x1=526.8956909179688 y1=84.27599334716797
chars=0-21 (len=21)
Załącznik nr 1 do SWZ
--------------------------------------------------------------------------------
[1] id=seg_p1_b1 page=1
bbox=page=1 x0=162.13999938964844 y0=86.69601440429688 x1=526.8956909179688 y1=112.39997863769531
chars=22-65 (len=43)
OPIS PRZEDMIOTU ZAMÓWIENIA DLA ZADANIA NR 1
--------------------------------------------------------------------------------
[2] id=seg_p1_b2 page=1
bbox=page=1 x0=261.7699890136719 y0=115.03999328613281 x1=336.15399169921875 y1=127.03999328613281
chars=66-80 (len=14)
(„OPZ zad. 1”)
--------------------------------

In [ ]:
import pickle
from pathlib import Path

CACHE_PATH = Path("opz_cache.pkl")

with open(CACHE_PATH, "rb") as f:
    loaded = pickle.load(f)

for name, value in loaded.items():
    globals()[name] = value

print("Załadowano z cache:", list(loaded.keys()))


# SIWZ Mapper – playground do testów pipeline'u

Ten notatnik służy do ręcznego „przeklikania” całego pipeline'u:
- PDFLoader,
- Segmenter,
- LLM + FakeGPT,
- VariantAggregator,
- ServiceMapper (stub),
- PDFExtractor,
- Pipeline,
- inspect_pdf.

Każda sekcja ma funkcje pomocnicze + przykładowe wywołania.


In [1]:
# Cell 1 – podstawowy setup

from pathlib import Path
import sys

# Jeśli źródła są w folderze src/, dodaj go do sys.path
PROJECT_ROOT = Path(".").resolve()
SRC_DIR = PROJECT_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("SRC_DIR in sys.path:", str(SRC_DIR) in sys.path)


PROJECT_ROOT: C:\Programowanie\SIWZ
SRC_DIR in sys.path: True


## Konfiguracja ścieżek do testowych PDF-ów

Ustaw tutaj ścieżki do kilku plików PDF, na których chcesz testować pipeline.


In [2]:
# Cell 1 – basic setup (ścieżki + sys.path)

from pathlib import Path
import sys

# Projekt zakładam, że odpalasz z katalogu głównego repo (tam, gdzie jest 'src' i np. 'scripts')
PROJECT_ROOT = Path(".").resolve()
SRC_DIR = PROJECT_ROOT / "src"

if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

print("PROJECT_ROOT:", PROJECT_ROOT)
print("SRC_DIR in sys.path:", str(SRC_DIR) in sys.path)

# Jedyny plik PDF do testów:
PDF_PATH = Path(
    r"C:\Programowanie\SIWZ\data\examples\OPZ_wybrane_strony.pdf"
)

print("PDF_PATH:", PDF_PATH)
print("PDF exists:", PDF_PATH.exists())


PROJECT_ROOT: C:\Programowanie\SIWZ
SRC_DIR in sys.path: True
PDF_PATH: C:\Programowanie\SIWZ\data\examples\OPZ_wybrane_strony.pdf
PDF exists: True


## Helpery: skracanie i podgląd segmentów


In [3]:
# Cell 2 – helpery

def preview_text(text: str, max_len: int = 200) -> str:
    t = text.replace("\n", " ⏎ ")
    return t if len(t) <= max_len else t[:max_len] + "…"

def print_segments_brief(segments, max_items: int = 20):
    print(f"Total segments: {len(segments)}\n")
    for s in segments[:max_items]:
        print(f"id={s.segment_id} page={s.page} len={len(s.text)} "
              f"chars={s.start_char}-{s.end_char}")
        print(preview_text(s.text))
        print("-" * 80)


## 1. PDFLoader – jak wyglądają bloki z Twojego PDF-a


In [ ]:
# Cell 3 – test PDFLoader na JEDNYM PDF

from siwz_mapper.io.pdf_loader import PDFLoader, load_pdf

def inspect_pdf_loader(pdf_path: Path, show_first_blocks: int = 30):
    blocks = load_pdf(pdf_path, extract_bboxes=True)
    print(f"PDF: {pdf_path}")
    print(f"Number of blocks: {len(blocks)}\n")

    for i, b in enumerate(blocks[:show_first_blocks]):
        print(f"[{i}] id={b.segment_id} page={b.page}")
        print(f"bbox={b.bbox}")
        print(f"chars={b.start_char}-{b.end_char} (len={len(b.text)})")
        print(preview_text(b.text))
        print("-" * 80)

    return blocks

blocks = inspect_pdf_loader(PDF_PATH, show_first_blocks=30)



PDF: C:\Programowanie\SIWZ\data\examples\OPZ_wybrane_strony.pdf
Number of blocks: 119

[0] id=seg_p1_b0 page=1
bbox=page=1 x0=423.30999755859375 y0=73.23599243164062 x1=526.8956909179688 y1=84.27599334716797
chars=0-21 (len=21)
Załącznik nr 1 do SWZ
--------------------------------------------------------------------------------
[1] id=seg_p1_b1 page=1
bbox=page=1 x0=162.13999938964844 y0=86.69601440429688 x1=526.8956909179688 y1=112.39997863769531
chars=22-65 (len=43)
OPIS PRZEDMIOTU ZAMÓWIENIA DLA ZADANIA NR 1
--------------------------------------------------------------------------------
[2] id=seg_p1_b2 page=1
bbox=page=1 x0=261.7699890136719 y0=115.03999328613281 x1=336.15399169921875 y1=127.03999328613281
chars=66-80 (len=14)
(„OPZ zad. 1”)
--------------------------------------------------------------------------------
[3] id=seg_p1_b3 page=1
bbox=page=1 x0=70.8239974975586 y0=129.6800079345703 x1=526.8842163085938 y1=185.71998596191406
chars=81-264 (len=183)
Świadczenie usług 

## 2. Segmenter – cięcie bloków na segmenty (na Twoim PDF-ie)


In [5]:
# Cell 5 – uruchomienie Segmenter na tym PDF

from siwz_mapper.preprocess.segmenter import Segmenter

def run_segmenter_on_main_pdf(pdf_path: Path, max_preview: int = 40):
    blocks = load_pdf(pdf_path, extract_bboxes=True)
    segmenter = Segmenter()
    segments = segmenter.segment(blocks)

    print(f"PDF: {pdf_path}")
    print(f"Num blocks:   {len(blocks)}")
    print(f"Num segments: {len(segments)}\n")

    print_segments_brief(segments, max_items=max_preview)
    return blocks, segments

blocks_seg, segments_seg = run_segmenter_on_main_pdf(PDF_PATH, max_preview=40)


PDF: C:\Programowanie\SIWZ\data\examples\OPZ_wybrane_strony.pdf
Num blocks:   119
Num segments: 155

Total segments: 155

id=seg_p1_b0 page=1 len=21 chars=0-21
Załącznik nr 1 do SWZ
--------------------------------------------------------------------------------
id=seg_p1_b1 page=1 len=43 chars=22-65
OPIS PRZEDMIOTU ZAMÓWIENIA DLA ZADANIA NR 1
--------------------------------------------------------------------------------
id=seg_p1_b2 page=1 len=14 chars=66-80
("OPZ zad. 1")
--------------------------------------------------------------------------------
id=seg_p1_b3 page=1 len=181 chars=81-262
Świadczenie usług grupowej opieki medycznej dla pracowników Zamawiającego oraz ⏎ członków ich rodzin w formie ubezpieczenia zdrowotnego lub abonamentu medycznego ⏎ w okresie 24 miesięcy
--------------------------------------------------------------------------------
id=seg_p1_b5_bullet0 page=1 len=23 chars=265-288
1. POSTANOWIENIA OGÓLNE
---------------------------------------------------------

In [6]:
def find_longest_by_text(objs):
    """
    Zwraca (najdłuższy_obiekt, długość_tekstu).
    Przyjmuje, że obiekt ma atrybut .text
    """
    if not objs:
        return None, 0
    
    longest = max(objs, key=lambda o: len(getattr(o, "text", "")))
    return longest, len(longest.text)


# Najdłuższy blok
longest_block, longest_block_len = find_longest_by_text(blocks)
print("=== NAJDŁUŻSZY BLOK ===")
print(f"Długość tekstu: {longest_block_len}")
# jeśli masz start_char / end_char / page, możesz też to pokazać:
if hasattr(longest_block, "page"):
    print(f"Strona: {longest_block.page}")
if hasattr(longest_block, "start_char") and hasattr(longest_block, "end_char"):
    print(f"Zakres znaków: {longest_block.start_char}–{longest_block.end_char}")
print("\nTreść bloku:\n")
print(longest_block.text)
print("\n" + "="*80 + "\n")

# Najdłuższy segment
longest_segment, longest_segment_len = find_longest_by_text(segments_seg)
print("=== NAJDŁUŻSZY SEGMENT ===")
print(f"Długość tekstu: {longest_segment_len}")
if hasattr(longest_segment, "page"):
    print(f"Strona: {getattr(longest_segment, 'page', 'N/A')}")
if hasattr(longest_segment, "start_char") and hasattr(longest_segment, "end_char"):
    print(f"Zakres znaków: {longest_segment.start_char}–{longest_segment.end_char}")
print("\nTreść segmentu:\n")
print(longest_segment.text)


=== NAJDŁUŻSZY BLOK ===
Długość tekstu: 1848
Strona: 17
Zakres znaków: 33948–35796

Treść bloku:

34.18. 
kości krzyżowo-ogonowej bok,  
34.19. 
kości nosa,  
34.20. 
kości piętowej,  
34.21. 
kości piętowej osiowe + bok,  
34.22. 
kości piętowej osiowe,  
34.23. 
kości piętowej bok, 
34.24. 
kości piętowych,  
34.25. 
kości podudzia AP,  
34.26. 
kości podudzia bok,  
34.27. 
kości podudzia AP + bok,  
34.28. 
kości obu podudzi (porównawcze),  
34.29. 
kości ramiennej AP,  
34.30. 
kości ramiennej bok,  
34.31. 
kości ramiennej AP + bok,  
34.32. 
kości skroniowej,  
34.33. 
kości udowej AP,  
34.34. 
kości udowej bok,  
34.35. 
kości udowej AP + bok,  
34.36. 
kręgosłupa – całego,  
34.37. 
kręgosłupa lędźwiowego AP,  
34.38. 
kręgosłupa lędźwiowego AP + bok,  
34.39. 
kręgosłupa lędźwiowego bok/skos,  
34.40. 
kręgosłupa lędźwiowego czynnościowe,  
34.41. 
kręgosłupa lędźwiowego-krzyżowego,  
34.42. 
kręgosłupa lędźwiowo-krzyżowego czynnościowe,  
34.43. 
kręgosłupa piersiowego AP, 

## 3. LLM


In [ ]:
import os

# 🔴 Najprościej (ale NIE wrzucaj tego do repo!)
os.environ["OPENAI_API_KEY"] = ""

# Kontrola:
print("OPENAI_API_KEY set:", bool(os.getenv("OPENAI_API_KEY")))

OPENAI_API_KEY set: True


In [8]:
# Cell 1 – importy i pomocnicza funkcja podglądu

from pathlib import Path
from collections import Counter

from siwz_mapper.io.pdf_loader import load_pdf
from siwz_mapper.preprocess.segmenter import Segmenter
from siwz_mapper.llm.gpt_client import GPTClient
from siwz_mapper.llm.classify_segments import classify_segments
from siwz_mapper.pipeline.variant_aggregator import VariantAggregator

def preview_text(text: str, max_len: int = 160) -> str:
    """Prosty skrót tekstu do logów."""
    if not text:
        return ""
    t = " ".join(str(text).split())
    return t if len(t) <= max_len else t[:max_len] + " [...]"


In [9]:
# Cell 2 – główna funkcja: PDF -> GPT -> VariantAggregator

def aggregate_variants_on_pdf_with_gpt(
    pdf_path: Path,
    model_name: str = "gpt-4o-mini",
    max_segments: int | None = None,
    show_progress: bool = True,
):
    """
    Pełny mini-flow:
    PDF -> PDFLoader -> Segmenter -> GPTClient -> VariantAggregator

    Zapisuje wyniki do globalnych zmiennych:
      last_blocks
      last_segments
      last_classifications
      last_updated_segments
      last_variant_groups

    Dzięki temu możesz później analizować wyniki bez ponownego wołania GPT.
    """
    global last_blocks, last_segments, last_classifications
    global last_updated_segments, last_variant_groups

    pdf_path = Path(pdf_path)
    print(f"=== Running pipeline on: {pdf_path} ===")

    # 1) Bloki + segmenty
    blocks = load_pdf(pdf_path, extract_bboxes=True)
    segmenter = Segmenter()
    segments = segmenter.segment(blocks)

    if max_segments is not None:
        segments_to_use = segments[:max_segments]
    else:
        segments_to_use = segments

    print(f"Num blocks   : {len(blocks)}")
    print(f"Num segments : {len(segments)}")
    print(f"Using        : {len(segments_to_use)} segment(s) for GPT\n")

    # 2) Klient GPT
    client = GPTClient(model=model_name, temperature=0.0)

    # 3) Klasyfikacja
    classifications = classify_segments(
        segments_to_use,
        client,
        show_progress=show_progress,
    )

    # 4) Agregacja wariantów
    aggregator = VariantAggregator()
    updated_segments, variant_groups = aggregator.aggregate(
        segments_to_use,
        classifications,
    )

    print(f"\nnum_segments_used = {len(segments_to_use)}")
    print(f"num_variants      = {len(variant_groups)}\n")

    for vg in variant_groups:
        pages = []
        if vg.header_segment:
            pages.append(vg.header_segment.page)
        pages.extend([s.page for s in vg.body_segments])
        pages.extend([s.page for s in vg.prophylaxis_segments])

        if pages:
            span = f"{min(pages)}–{max(pages)}"
        else:
            span = "[no pages]"

        print(f"Variant {vg.variant_id}")
        print(f"  pages span   : {span}")
        print(f"  body         : {len(vg.body_segments)} segments")
        print(f"  prophylaxis  : {len(vg.prophylaxis_segments)} segments")
        if vg.header_segment:
            print("  header preview:",
                  preview_text(vg.header_segment.text, max_len=120))
        else:
            print("  header preview: [None]")
        print("-" * 80)

    # 5) zapis do globalnych zmiennych (do dalszej analizy)
    last_blocks = blocks
    last_segments = segments_to_use
    last_classifications = classifications
    last_updated_segments = updated_segments
    last_variant_groups = variant_groups

    return updated_segments, variant_groups


In [10]:
# Cell 3 – helper do szybkiego podsumowania wariantów

def summarize_last_variants():
    """
    Zwięzłe podsumowanie wariantów z ostatniego uruchomienia
    aggregate_variants_on_pdf_with_gpt.
    """
    try:
        variant_groups = last_variant_groups
    except NameError:
        print("Brak last_variant_groups – uruchom najpierw aggregate_variants_on_pdf_with_gpt().")
        return

    print("=== Summary of last_variant_groups ===\n")
    for vg in variant_groups:
        pages = []
        if vg.header_segment:
            pages.append(vg.header_segment.page)
        pages.extend([s.page for s in vg.body_segments])
        pages.extend([s.page for s in vg.prophylaxis_segments])

        span = f"{min(pages)}–{max(pages)}" if pages else "[no pages]"

        print(f"Variant {vg.variant_id}")
        print(f"  pages span     : {span}")
        print(f"  body segments  : {len(vg.body_segments)}")
        print(f"  prophylaxis seg: {len(vg.prophylaxis_segments)}")
        if vg.header_segment:
            print("  header preview:",
                  preview_text(vg.header_segment.text, max_len=160))
        print("-" * 70)


def show_last_variant_examples(variant_id: str, max_body: int = 5, max_proph: int = 5):
    """
    Pokaż przykładowe segmenty body/prophylaxis dla wybranego wariantu.
    """
    try:
        variant_groups = last_variant_groups
    except NameError:
        print("Brak last_variant_groups – uruchom najpierw aggregate_variants_on_pdf_with_gpt().")
        return

    vg = None
    for g in variant_groups:
        if g.variant_id == variant_id:
            vg = g
            break

    if vg is None:
        print(f"Nie znaleziono wariantu {variant_id}")
        return

    print(f"=== Examples for {vg.variant_id} ===")

    if vg.header_segment:
        print("\n--- HEADER ---")
        print(f"({vg.header_segment.segment_id}, page {vg.header_segment.page})")
        print(preview_text(vg.header_segment.text, max_len=200))
    else:
        print("\n[No header_segment]")

    print("\n--- BODY (first) ---")
    for s in vg.body_segments[:max_body]:
        print(f"({s.segment_id}, page {s.page})")
        print(preview_text(s.text, max_len=200))
        print("-" * 40)

    print("\n--- PROPHYLAXIS (first) ---")
    if vg.prophylaxis_segments:
        for s in vg.prophylaxis_segments[:max_proph]:
            print(f"({s.segment_id}, page {s.page})")
            print(preview_text(s.text, max_len=200))
            print("-" * 40)
    else:
        print("[No prophylaxis segments in this variant]")


In [10]:
# Cell 4 – przykład użycia na małym PDF-ie

SMALL_PDF_PATH = Path(
    r"C:\Programowanie\SIWZ\data\examples\OPZ_wybrane_strony.pdf"
)  # <-- podmień na swoją ścieżkę

updated_segments_small, variant_groups_small = aggregate_variants_on_pdf_with_gpt(
    SMALL_PDF_PATH,
    model_name="gpt-4o-mini",
    max_segments=None,   # na razie pełny dokument; w razie czego można ograniczyć
    show_progress=True,
)

summarize_last_variants()

# Jeśli chcesz podejrzeć konkretny wariant:
show_last_variant_examples("V1")
show_last_variant_examples("V2")


=== Running pipeline on: C:\Programowanie\SIWZ\data\examples\OPZ_wybrane_strony.pdf ===
Num blocks   : 119
Num segments : 155
Using        : 155 segment(s) for GPT


num_segments_used = 155
num_variants      = 2

Variant V1
  pages span   : 3–10
  body         : 41 segments
  prophylaxis  : 2 segments
  header preview: WARIANT 1
--------------------------------------------------------------------------------
Variant V2
  pages span   : 11–20
  body         : 48 segments
  prophylaxis  : 7 segments
  header preview: WARIANT 2
--------------------------------------------------------------------------------
=== Summary of last_variant_groups ===

Variant V1
  pages span     : 3–10
  body segments  : 41
  prophylaxis seg: 2
  header preview: WARIANT 1
----------------------------------------------------------------------
Variant V2
  pages span     : 11–20
  body segments  : 48
  prophylaxis seg: 7
  header preview: WARIANT 2
----------------------------------------------------------------

In [11]:
# Cell X – index segmentów i klasyfikacji z ostatniego runu

from typing import Dict, Tuple
from siwz_mapper.llm.classify_segments import SegmentClassification
from siwz_mapper.models import PdfSegment

def build_last_segment_index() -> Dict[str, Tuple[PdfSegment, SegmentClassification]]:
    """
    Buduje słownik:
        segment_id -> (segment, classification)
    na podstawie ostatniego wywołania aggregate_variants_on_pdf_with_gpt.
    """
    try:
        segments = last_segments
        classifications = last_classifications
    except NameError:
        print("Brak last_segments / last_classifications – uruchom najpierw aggregate_variants_on_pdf_with_gpt().")
        return {}

    if len(segments) != len(classifications):
        print("Uwaga: różna długość segments i classifications – coś jest nie tak.")
        return {}

    index = {
        s.segment_id: (s, c)
        for s, c in zip(segments, classifications)
    }

    print(f"Zbudowano index dla {len(index)} segmentów.")
    return index


In [12]:
# Cell A – surowa ekstrakcja usług z wariantów (blok_category = 'other')

from siwz_mapper.pipeline.service_extractor import ServiceExtractor
from siwz_mapper.models import VariantServiceItem

try:
    variant_groups = last_variant_groups
except NameError:
    raise RuntimeError(
        "Brak last_variant_groups – uruchom najpierw aggregate_variants_on_pdf_with_gpt()."
    )

extractor = ServiceExtractor()
variant_items_by_id: dict[str, list[VariantServiceItem]] = extractor.extract_from_variants(variant_groups)

print("Dostępne warianty:", list(variant_items_by_id.keys()))
for vid, items in variant_items_by_id.items():
    print(vid, "=>", len(items), "services")

Dostępne warianty: ['V1', 'V2']
V1 => 345 services
V2 => 440 services


In [13]:
from collections import defaultdict

def list_blocks_for_variant(variant_id: str):
    """
    Wypisz listę bloków (block_no) dla danego wariantu z krótkim podglądem nagłówka
    i liczbą usług w bloku.
    """
    items = variant_items_by_id.get(variant_id)
    if not items:
        print(f"Brak usług dla wariantu {variant_id}")
        return

    by_block: dict[str, list[VariantServiceItem]] = defaultdict(list)
    for it in items:
        key = it.block_no or "_none"
        by_block[key].append(it)

    for block_no in sorted(by_block.keys(), key=lambda x: (x == "_none", x)):
        block_items = by_block[block_no]
        heading = None
        for it in block_items:
            if it.block_heading_raw:
                heading = it.block_heading_raw
                break

        print(f"=== Block {block_no} (items={len(block_items)}) ===")
        if heading:
            print("  heading:", heading)
        else:
            print("  heading: [brak]")
        print("-" * 60)


def show_block_details(variant_id: str, block_no: str):
    """
    Pokaż szczegóły jednego bloku: nagłówek + wszystkie VariantServiceItem
    z service_local_id / service_text / continuation.
    """
    items = variant_items_by_id.get(variant_id)
    if not items:
        print(f"Brak usług dla wariantu {variant_id}")
        return

    block_items = [it for it in items if it.block_no == block_no]

    if not block_items:
        print(f"Brak itemów w bloku {block_no} dla wariantu {variant_id}")
        return

    heading = None
    for it in block_items:
        if it.block_heading_raw:
            heading = it.block_heading_raw
            break

    print(f"=== {variant_id} / block {block_no} ===")
    if heading:
        print("HEADING:", heading)
    print()

    for it in block_items:
        print(f"- local_id={it.service_local_id!r}  page={it.page}")
        print(f"  text     = {it.service_text}")
        if it.extra.get("continuation_lines"):
            print("  continuation:")
            for ln in it.extra["continuation_lines"]:
                print("    >", ln)
        print("  source_segment_id:", it.source_segment_id)
        print("  block_category   :", it.block_category)
        print("-" * 50)


def browse_blocks_interactive(variant_id: str):
    """
    Bardzo prosty "przeklikiwacz": pokazuje bloki po kolei; ENTER = następny, q = wyjście.
    """
    items = variant_items_by_id.get(variant_id)
    if not items:
        print(f"Brak usług dla wariantu {variant_id}")
        return

    from collections import OrderedDict
    order = OrderedDict()
    for it in items:
        key = it.block_no or "_none"
        if key not in order:
            order[key] = None

    block_list = list(order.keys())

    print(f"Wariant {variant_id} – dostępne bloki:", block_list)
    print("ENTER = następny blok, 'q' + ENTER = wyjście\n")

    for block_no in block_list:
        if block_no == "_none":
            print(">>> Blok _none (itemy bez block_no) – pomijam")
            continue

        show_block_details(variant_id, block_no)
        cmd = input("Dalej? [ENTER/q] ")
        if cmd.strip().lower().startswith("q"):
            break


In [14]:
list_blocks_for_variant("V1")

=== Block 1 (items=1) ===
  heading: Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.
------------------------------------------------------------
=== Block 10 (items=18) ===
  heading: Konsultacyjne zabiegi ambulatoryjne z zakresu okulistyki (na podstawie skierowania lekarskiego):
------------------------------------------------------------
=== Block 11 (items=4) ===
  heading: Konsultacyjne zabiegi ambulatoryjne z zakresu urologii (na podstawie skierowania lekarskiego):
------------------------------------------------------------
=== Block 12 (items=7) ===
  heading: Konsultacyjne zabiegi ambulatoryjne z zakresu laryngologii (na podstawie skierowania
------------------------------------------------------------
=== Block 13 (items=5) ===
  heading: Ambulatoryjna opieka pielęgniarska (na podstawie skierowania lekarskiego):
------------------------------------------------------------
=== Block 14 (items=37) ===
  heading: Badania biochemiczne z krw

In [15]:
from siwz_mapper.pipeline.service_extractor import ServiceExtractor

extractor = ServiceExtractor()

def debug_line_stream_for_variant(variant_id: str, max_lines: int = 200):
    """
    Pokaż surowy line_stream, na którym działa ServiceExtractor,
    dla wybranego wariantu.
    """
    try:
        variant_groups = last_variant_groups
    except NameError:
        raise RuntimeError("Brak last_variant_groups – uruchom najpierw aggregate_variants_on_pdf_with_gpt().")

    vg = None
    for g in variant_groups:
        if g.variant_id == variant_id:
            vg = g
            break

    if vg is None:
        print(f"Nie znaleziono wariantu {variant_id}")
        return

    line_stream = extractor._build_line_stream(vg)

    print(f"=== line_stream for variant {variant_id} (len={len(line_stream)}) ===\n")
    for i, ln in enumerate(line_stream[:max_lines]):
        seg = ln["segment"]
        flag = "P" if ln["is_prophylaxis"] else "B"
        print(
            f"[{i:03}] page={seg.page:2d} seg={seg.segment_id:12s} "
            f"kind={flag} line_idx={ln['line_idx_in_segment']:2d}  -> {ln['text']!r}"
        )


In [16]:
def debug_lines_with_prefix(variant_id: str, prefix: str):
    """
    Pokaż linie w line_stream, które zaczynają się od danego prefiksu
    (np. '4.', '11.', '15.', '21.').
    """
    try:
        variant_groups = last_variant_groups
    except NameError:
        raise RuntimeError("Brak last_variant_groups – uruchom najpierw aggregate_variants_on_pdf_with_gpt().")

    vg = next(g for g in last_variant_groups if g.variant_id == variant_id)
    line_stream = extractor._build_line_stream(vg)

    print(f"=== lines for {variant_id} starting with {prefix!r} ===\n")
    found = False
    for ln in line_stream:
        txt = ln["text"].lstrip()  # obcinamy leading spaces
        if txt.startswith(prefix):
            found = True
            seg = ln["segment"]
            print(
                f"page={seg.page} seg={seg.segment_id} "
                f"line_idx={ln['line_idx_in_segment']} -> {ln['text']!r}"
            )
    if not found:
        print("NIC nie znalazłem – wygląda na to, że ta linia nie jest w line_stream.")


In [17]:
debug_lines_with_prefix("V1", "4.")
debug_lines_with_prefix("V1", "10.")
debug_lines_with_prefix("V1", "11.")
debug_lines_with_prefix("V1", "12.")
debug_lines_with_prefix("V1", "13.")
debug_lines_with_prefix("V1", "25.")

=== lines for V1 starting with '4.' ===

page=3 seg=seg_p3_b3 line_idx=4 -> '4. Nielimitowany dostęp do konsultacji medycznych świadczonych przez następujących'
page=3 seg=seg_p3_b5 line_idx=0 -> '4.1. lekarz podstawowej opieki medycznej/lekarz rodzinny'
page=3 seg=seg_p3_b5 line_idx=1 -> '4.2. internista'
page=3 seg=seg_p3_b5 line_idx=2 -> '4.3. pediatra'
page=3 seg=seg_p3_b5 line_idx=3 -> '4.4. chirurg ogólny'
page=3 seg=seg_p3_b5 line_idx=4 -> '4.5. dermatolog'
page=3 seg=seg_p3_b5 line_idx=5 -> '4.6. dermatolog wenerolog'
page=3 seg=seg_p3_b5 line_idx=6 -> '4.7. ginekolog/ginekolog-położnik'
page=3 seg=seg_p3_b5 line_idx=7 -> '4.8. otolaryngolog'
page=3 seg=seg_p3_b5 line_idx=8 -> '4.9. neurolog'
page=3 seg=seg_p3_b5 line_idx=9 -> '4.10. neurochirurg'
page=3 seg=seg_p3_b5 line_idx=11 -> '4.11. okulista'
page=3 seg=seg_p3_b5 line_idx=13 -> '4.12. pulmonolog'
page=3 seg=seg_p3_b5 line_idx=15 -> '4.13. reumatolog'
page=3 seg=seg_p3_b5 line_idx=17 -> '4.14. urolog'
page=3 seg=seg_p3_b5

In [24]:
def debug_segment_text(segment_id: str):
    try:
        segments = last_segments
        classifications = last_classifications
    except NameError:
        raise RuntimeError("Brak last_segments / last_classifications – uruchom najpierw aggregate_variants_on_pdf_with_gpt().")

    for seg, cls in zip(segments, classifications):
        if seg.segment_id == segment_id:
            print(f"segment_id = {seg.segment_id}")
            print(f"page       = {seg.page}")
            print(f"label      = {cls.label}")
            print("-" * 60)
            print(seg.text)
            print("-" * 60)
            return

    print(f"Nie znalazłem segmentu {segment_id} w last_segments.")
debug_segment_text("seg_p5_b0_p1")        # tam jest 15.
debug_segment_text("seg_p6_b2_split0")    # tam jest 21.


segment_id = seg_p5_b0_p1
page       = 5
label      = variant_body
------------------------------------------------------------
15.
Diagnostyka miażdżycy – badania z krwi (na podstawie skierowania lekarskiego):
15.1.
cholesterol – badanie paskowe,
15.2.
cholesterol całkowity (CHOL),
15.3.
cholesterol HDL,
15.4.
cholesterol LDL,
15.5.
lipidogram (TC, HDL, LDL, TRG/TG),
15.6.
triglicerydy (TRIG/TG).
16. Diagnostyka cukrzycy – badania z krwi (na podstawie skierowania lekarskiego):
------------------------------------------------------------
segment_id = seg_p6_b2_split0
page       = 6
label      = variant_body
------------------------------------------------------------
20.1. amylaza/diastaza (AMY),
20.2. amylaza/diastaza z dobowej zbiórki moczu,
20.3. badanie ogólne moczu,
20.4. badanie ogólne moczu – badanie paskowe,
20.5. białko ilościowe,
20.6. białko z dobowej zbiórki moczu,
20.7. białko Bence-Jonesa metodą,
20.8. immunofiksacji,
20.9. bilirubina,
20.10. glukoza/cukier i ketony,
20.1

In [19]:
# Cell B – konfiguracja kategorii + classifier bloków (LLM)

from siwz_mapper.llm.gpt_client import GPTClient
from siwz_mapper.llm.service_block_classifier import (
    ServiceCategoryDef,
    ServiceBlockCategoryClassifier,
)

# 1) Konfigurowalne kategorie (na początek prosto – potem możesz przenieść do JSON/YAML)
categories = [
    ServiceCategoryDef(
        id="consultation",
        label="Konsultacje lekarskie",
        description="Wizyty u lekarzy POZ i specjalistów, porady, konsultacje.",
        examples=["wizyta u internisty", "konsultacja kardiologiczna"],
    ),
    ServiceCategoryDef(
        id="lab",
        label="Diagnostyka laboratoryjna",
        description="Badania krwi, moczu, kału, testy serologiczne itd.",
        examples=["morfologia krwi", "badanie ogólne moczu"],
    ),
    ServiceCategoryDef(
        id="imaging",
        label="Diagnostyka obrazowa",
        description="USG, RTG, TK, rezonans, mammografia itd.",
        examples=["RTG klatki piersiowej", "USG jamy brzusznej"],
    ),
    ServiceCategoryDef(
        id="prophylaxis",
        label="Profilaktyka / szczepienia / przeglądy",
        description="Szczepienia, przeglądy stanu zdrowia, programy profilaktyczne.",
        examples=["szczepienie przeciw grypie", "profilaktyczny przegląd stanu zdrowia"],
    ),
    # Możesz spokojnie dorzucać kolejne, np. 'rehab', 'telemedicine' itd.
]

# Klient GPT – jak w reszcie projektu
gpt_client = GPTClient(model="gpt-4o-mini", temperature=0.0)

block_classifier = ServiceBlockCategoryClassifier(
    client=gpt_client,
    categories=categories,
)


In [21]:
# Cell C – klasyfikacja bloków przez LLM -> block_category_map

# 1) Konteksty bloków (z surowych items, gdzie block_category = 'other')
block_contexts = block_classifier.build_block_contexts(
    items_by_variant=variant_items_raw,
    max_examples_per_block=8,   # ile usług pokazujemy LLM-owi jako przykłady
)
print(f"Liczba bloków do sklasyfikowania: {len(block_contexts)}")

# 2) Wołanie LLM – dla każdego bloku wybiera category_id
block_cats = block_classifier.classify_blocks(block_contexts)

# 3) Mapowanie category_id -> BlockCategory (Literal w models.py)
from siwz_mapper.models import BlockCategory

category_id_to_block_category: dict[str, BlockCategory] = {
    "consultation": "consultation",
    "lab": "lab",
    "imaging": "imaging",
    "prophylaxis": "prophylaxis",
    # wszystkie inne (nieznane) wylecą na 'other'
}

block_category_map = block_classifier.to_block_category_map(
    block_cats,
    category_id_to_block_category=category_id_to_block_category,
)

print("Przykładowe mapowania (pierwsze kilka):")
for (vid, bno), cat in list(block_category_map.items())[:10]:
    print(f"  ({vid}, block {bno}) -> {cat}")


Liczba bloków do sklasyfikowania: 53
Przykładowe mapowania (pierwsze kilka):
  (V1, block ?) -> consultation
  (V1, block 6) -> consultation
  (V1, block 7) -> consultation
  (V1, block 8) -> consultation
  (V1, block 10) -> consultation
  (V1, block 12) -> consultation
  (V1, block 14) -> lab
  (V1, block 16) -> lab
  (V1, block 18) -> lab
  (V1, block 19) -> lab


In [22]:
# Cell D – ekstrakcja usług z użyciem block_category_map (LLM-driven)

from siwz_mapper.pipeline.service_extractor import ServiceExtractor

extractor = ServiceExtractor(block_category_map=block_category_map)
variant_items_final: dict[str, list[VariantServiceItem]] = extractor.extract_for_variants(variant_groups)

print("Podsumowanie liczby usług po kategoryzacji bloków:")
for vid, items in variant_items_final.items():
    print(f"  {vid}: {len(items)} services")


Podsumowanie liczby usług po kategoryzacji bloków:
  V1: 341 services
  V2: 442 services


In [26]:
list_blocks_for_variant("V1")
show_block_details("V1", "4")   # Twoje "Nielimitowany dostęp..."
browse_blocks_interactive("V1")


NameError: name 'variant_items_by_id' is not defined

In [23]:
# Cell E – podgląd: ile usług na wariant + kilka losowych usług z V1/V2

import random

def summarize_variant_items(variant_items: dict[str, list[VariantServiceItem]]):
    print("=== Liczba usług per wariant ===")
    for vid, items in variant_items.items():
        # policz też ile w każdej kategorii
        by_cat: dict[str, int] = {}
        for it in items:
            by_cat[it.block_category] = by_cat.get(it.block_category, 0) + 1

        cats_str = ", ".join(f"{cat}: {cnt}" for cat, cnt in by_cat.items())
        print(f"{vid}: {len(items)} services (kategorie: {cats_str})")

def show_random_examples(
    vid: str,
    n: int = 10,
    only_category: str | None = None,
):
    items = variant_items_final.get(vid, [])
    if only_category is not None:
        items = [it for it in items if it.block_category == only_category]

    if not items:
        print(f"[{vid}] Brak usług (filter category={only_category!r}).")
        return

    sample = random.sample(items, k=min(n, len(items)))
    print(f"\n=== Losowe usługi dla {vid} (n={len(sample)}, category={only_category}) ===")
    for it in sample:
        lid = it.service_local_id or "–"
        print(
            f"[page {it.page}] block {it.block_no or '?'} "
            f"({it.block_category}) [{lid}] {it.service_text}"
        )

# Użycie:
summarize_variant_items(variant_items_final)
show_random_examples("V1", n=10)
show_random_examples("V2", n=10)

# Jeśli chcesz podejrzeć tylko np. konsultacje:
# show_random_examples("V1", n=10, only_category="consultation")
# show_random_examples("V2", n=10, only_category="lab")


=== Liczba usług per wariant ===
V1: 341 services (kategorie: other: 18, consultation: 46, lab: 120, imaging: 144, prophylaxis: 13)
V2: 442 services (kategorie: other: 27, consultation: 70, lab: 158, imaging: 174, prophylaxis: 13)

=== Losowe usługi dla V1 (n=10, category=None) ===
[page 5] block 14 (lab) [14.35] wapń całkowity (Ca),
[page 3] block ? (other) [4.12] pulmonolog
[page 5] block 14 (lab) [14.20] białko C-reaktywne (CRP/hs-CRP) – jakościowo,
[page 9] block 27 (imaging) [27.5] jąder i najądrzy,
[page 7] block 23 (imaging) [23.43] kręgosłupa piersiowego AP,
[page 9] block 23 (imaging) [23.108] stopy AP + bok,
[page 7] block 23 (imaging) [23.21] kości piętowej osiowe + bok,
[page 8] block 23 (imaging) [23.58] łopatki osiowe,
[page 4] block 12 (consultation) [–] Kisselbacha),
[page 7] block 23 (imaging) [23.8] czaszki półosiowe wg Orleya/potylicy wg Towna,

=== Losowe usługi dla V2 (n=10, category=None) ===
[page 14] block 24 (lab) [24.12] morfologia krwi obwodowej z rozmazem, o

In [24]:
variant_items_final

{'V1': [VariantServiceItem(variant_id='V1', block_no=None, block_heading_raw=None, block_category='other', service_local_id='4.1', service_text='lekarz podstawowej opieki medycznej/lekarz rodzinny', is_prophylaxis=False, is_occupational_medicine=False, is_telemedicine=False, is_pricing_only=False, source_segment_id='seg_p3_b5', page=3, extra={'source_line': '4.1. lekarz podstawowej opieki medycznej/lekarz rodzinny'}),
  VariantServiceItem(variant_id='V1', block_no=None, block_heading_raw=None, block_category='other', service_local_id='4.2', service_text='internista', is_prophylaxis=False, is_occupational_medicine=False, is_telemedicine=False, is_pricing_only=False, source_segment_id='seg_p3_b5', page=3, extra={'source_line': '4.2. internista'}),
  VariantServiceItem(variant_id='V1', block_no=None, block_heading_raw=None, block_category='other', service_local_id='4.3', service_text='pediatra', is_prophylaxis=False, is_occupational_medicine=False, is_telemedicine=False, is_pricing_only=F

In [28]:
import pandas as pd
from pathlib import Path

def load_category_vocab_from_excel(xlsx_path: Path) -> list[dict]:
    """
    Czyta kody_mapowanie.xlsx i zwraca listę unikalnych par
    (Kategoria, Podkategoria) jako słowniki:
       {"id": "...", "category": "...", "subcategory": "..."}
    """
    df = pd.read_excel(xlsx_path)

    # spróbujmy znaleźć kolumny po nazwie (na wszelki wypadek ignorujemy wielkość liter)
    cols = {c.lower(): c for c in df.columns}
    col_k = cols.get("kategoria", None)
    col_p = cols.get("podkategoria", None)

    if col_k is None or col_p is None:
        raise ValueError(f"Nie znaleziono kolumn 'Kategoria' i 'Podkategoria' w {xlsx_path}")

    df = df[[col_k, col_p]].copy()
    df[col_k] = df[col_k].astype(str).str.strip()
    df[col_p] = df[col_p].astype(str).str.strip()

    df_unique = df.drop_duplicates().reset_index(drop=True)

    out = []
    for _, row in df_unique.iterrows():
        kat = row[col_k] or ""
        pod = row[col_p] or ""
        if pod and pod.lower() != "nan":
            cat_id = f"{kat}__{pod}"
        else:
            cat_id = kat
            pod = None

        out.append(
            {
                "id": cat_id,
                "category": kat,
                "subcategory": pod,
            }
        )
    return out

excel_path = Path(r"C:\Programowanie\SIWZ\data\examples\kody_mapowanie.xlsx")
category_vocab = load_category_vocab_from_excel(excel_path)
print("Liczba unikalnych kategorii (Kategoria x Podkategoria):", len(category_vocab))
category_vocab[:5]


Liczba unikalnych kategorii (Kategoria x Podkategoria): 941


[{'id': 'Badania laboratoryjne i inne__Analityka',
  'category': 'Badania laboratoryjne i inne',
  'subcategory': 'Analityka'},
 {'id': 'Badania laboratoryjne i inne__Morfologia krwi i badania hematologiczne',
  'category': 'Badania laboratoryjne i inne',
  'subcategory': 'Morfologia krwi i badania hematologiczne'},
 {'id': 'Badania laboratoryjne i inne__Serologia grup krwi',
  'category': 'Badania laboratoryjne i inne',
  'subcategory': 'Serologia grup krwi'},
 {'id': 'Badania laboratoryjne i inne__Wirusologia',
  'category': 'Badania laboratoryjne i inne',
  'subcategory': 'Wirusologia'},
 {'id': 'Badania laboratoryjne i inne__Badania układu krzepnięcia',
  'category': 'Badania laboratoryjne i inne',
  'subcategory': 'Badania układu krzepnięcia'}]

In [3]:
from siwz_mapper.eval.manual_items import ManualDocVariantItems

path = r"C:\Programowanie\SIWZ\data\manual\OPZ_manual_variant_items.json"  # <-- podaj swój JSON
doc = ManualDocVariantItems.load(path)

print(doc.doc_id)
print(doc.pdf_path)
print(doc.variant_items_by_id.keys())
print(len(doc.variant_items_by_id["V1"]))


OPZ_wybrane_strony
C:\Programowanie\SIWZ\data\examples\OPZ_wybrane_strony.pdf
dict_keys(['V1'])
331


In [4]:
from siwz_mapper.eval.strategies import DummyHeuristicStrategy

s = DummyHeuristicStrategy()
print(s.name)


dummy_heuristic


In [5]:
from siwz_mapper.eval.manual_items import ManualDocVariantItems
from siwz_mapper.eval.strategies import DummyHeuristicStrategy
from siwz_mapper.eval.runner import run_strategy_on_manual_doc

# 1. Wczytaj swój JSON z manualnymi pozycjami
json_path = r"C:\Programowanie\SIWZ\data\manual\OPZ_manual_variant_items.json"  # tu Twój plik
manual_doc = ManualDocVariantItems.load(json_path)

print("Doc:", manual_doc.doc_id)

# 2. Utwórz strategię
strategy = DummyHeuristicStrategy()

# 3. Odpal strategię
updated_doc = run_strategy_on_manual_doc(manual_doc, strategy)

# 4. Zobacz pierwszą pozycję
first_variant_id = next(iter(updated_doc.variant_items_by_id.keys()))
first_item = updated_doc.variant_items_by_id[first_variant_id][0]

print("Variant:", first_item.variant_id)
print("Service local id:", first_item.service_local_id)
print("Text:", first_item.service_text)
print("Predicted services:", first_item.predicted_services)


Doc: OPZ_wybrane_strony
Variant: V1
Service local id: 1
Text: Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.
Predicted services: []


In [6]:
from siwz_mapper.eval.codebook import load_service_codes_from_excel

excel_path = r"C:\Programowanie\SIWZ\data\examples\kody_mapowanie.xlsx"
codes = load_service_codes_from_excel(excel_path)

print("Loaded codes:", len(codes))
print(codes[0])
print(codes[0].as_prompt_line())


Loaded codes: 5291
code='AAAA01' category='Badania laboratoryjne i inne' subcategory='Analityka' name='Badanie ogólne moczu (profil)'
AAAA01: [Badania laboratoryjne i inne / Analityka] Badanie ogólne moczu (profil)


In [ ]:
import os
# 🔴 Najprościej (ale NIE wrzucaj tego do repo!)
os.environ["OPENAI_API_KEY"] = ""


In [ ]:
from siwz_mapper.eval.manual_items import ManualDocVariantItems
from siwz_mapper.eval.codebook import load_service_codes_from_excel
from siwz_mapper.eval.strategies import SingleLLMMappingStrategyV1
from siwz_mapper.eval.runner import run_strategy_on_json_path
from siwz_mapper.llm.gpt_client import GPTClient  # Twój realny client


# 1. Ścieżki
json_path = r"C:\Programowanie\SIWZ\data\examples\OPZ_manual_variant_items.json"
excel_path = r"C:\Programowanie\SIWZ\data\examples\kody_mapowanie.xlsx"
output_path = r"C:\Programowanie\SIWZ\data\examples\OPZ_wybrane_strony_with_predictions.json"

# 1. Ładujemy kody z Excela
service_codes = load_service_codes_from_excel(excel_path)

# 2. Tworzymy klienta LLM
client = GPTClient(model="gpt-5-mini", temperature=1)

# 3. Strategia LLM
strategy = SingleLLMMappingStrategyV1(
    client=client,
    service_codes=service_codes,
)

# 4. Ile pozycji chcesz przetworzyć?
c = 10  # np. pierwsze 10 pozycji w JSON

updated_doc = run_strategy_on_json_path(
    json_path=json_path,
    strategy=strategy,
    output_path=output_path,
    max_items=c,
    show_progress=True,
)


[1/10] variant=V1, service_local_id=1
[2/10] variant=V1, service_local_id=2.1
[3/10] variant=V1, service_local_id=2.2
[4/10] variant=V1, service_local_id=3
[5/10] variant=V1, service_local_id=4.1
[6/10] variant=V1, service_local_id=4.2
[7/10] variant=V1, service_local_id=4.3
[8/10] variant=V1, service_local_id=4.4
[9/10] variant=V1, service_local_id=4.5
[10/10] variant=V1, service_local_id=4.6


In [4]:
# Ile wariantów?
print("Variants in doc:", list(updated_doc.variant_items_by_id.keys()))

# Weźmy pierwszy wariant
first_variant_id = next(iter(updated_doc.variant_items_by_id.keys()))
items = updated_doc.variant_items_by_id[first_variant_id]

print(f"Liczba pozycji w wariancie {first_variant_id}:", len(items))

# Podejrzyjmy pierwsze np. 5 pozycji
for item in items[:5]:
    print("\n---")
    print(f"service_local_id: {item.service_local_id}")
    print(f"Tekst: {item.service_text}")
    print(f"Nagłówek: {item.block_heading_raw}")
    
    if item.predicted_services:
        codes = [c.code for c in item.predicted_services]
        labels = [c.label for c in item.predicted_services]
        print("Predicted codes:", codes)
        print("Labels:", labels)
    else:
        print("Predicted codes: BRAK (None/[])")
updated_doc

Variants in doc: ['V1']
Liczba pozycji w wariancie V1: 331

---
service_local_id: 1
Tekst: Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.
Nagłówek: Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.
Predicted codes: BRAK (None/[])

---
service_local_id: 2.1
Tekst: za pośrednictwem infolinii dostępnej całą dobę,
Nagłówek: Rezerwacja wizyt:
Predicted codes: BRAK (None/[])

---
service_local_id: 2.2
Tekst: przez Internet (wykonawca udostępni odpowiednią stronę internetową/formularz),
Nagłówek: Rezerwacja wizyt:
Predicted codes: BRAK (None/[])

---
service_local_id: 3
Tekst: Możliwość otrzymywania SMS z potwierdzeniem terminu świadczeń zdrowotnych.
Nagłówek: Możliwość otrzymywania SMS z potwierdzeniem terminu świadczeń zdrowotnych.
Predicted codes: BRAK (None/[])

---
service_local_id: 4.1
Tekst: lekarz podstawowej opieki medycznej/lekarz rodzinny
Nagłówek: Nielimitowany dostęp do konsultacji medycznych świadczon

ManualDocVariantItems(doc_id='OPZ_wybrane_strony', pdf_path='C:\\Programowanie\\SIWZ\\data\\examples\\OPZ_wybrane_strony.pdf', variant_items_by_id={'V1': [VariantItem(variant_id='V1', block_no='1', block_heading_raw='Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.', block_category='other', service_local_id='1', service_text='Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.', is_prophylaxis=False, is_occupational_medicine=False, is_telemedicine=False, is_pricing_only=False, source_segment_id='manual_p7_1_r2', page=7, extra=VariantItemExtra(from_manual=True, block_heading_raw='Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.'), expected_codes=None, predicted_services=[], final_codes=None), VariantItem(variant_id='V1', block_no='2', block_heading_raw='Rezerwacja wizyt:', block_category='other', service_local_id='2.1', service_text='za pośrednictwem infolinii dostępnej całą

In [4]:
from siwz_mapper.eval.codebook import load_service_codes_from_excel
from siwz_mapper.eval.strategies import SingleLLMMappingStrategyV2
from siwz_mapper.eval.runner import run_strategy_on_json_path
from siwz_mapper.llm.gpt_client import GPTClient

json_path = r"C:\Programowanie\SIWZ\data\examples\OPZ_manual_variant_items.json"
excel_path = r"C:\Programowanie\SIWZ\data\examples\kody_mapowanie.xlsx"
output_path = r"C:\Programowanie\SIWZ\data\examples\OPZ_wybrane_strony_with_predictions_v2.json"
service_codes = load_service_codes_from_excel(excel_path)
client = GPTClient(model="gpt-5-mini", temperature=1)

strategy = SingleLLMMappingStrategyV2(
    client=client,
    service_codes=service_codes,
    debug=True,              # pokaże statystyki promptu raz
    debug_log_prompts=True,  # pokaże pełny prompt + odpowiedź
    debug_max_items=1,       # tylko dla pierwszej pozycji
)

updated_doc, stats = run_strategy_on_json_path(
    json_path=json_path,
    strategy=strategy,
    output_path=output_path,
    max_items=5,
    show_progress=True,
)


[DEBUG] SingleLLMMappingStrategyV2 prompt stats:
  service_text chars: 87
  codes in prompt   : 5143
  codes_block chars : 342520

[DEBUG PROMPT V2 #1] service_local_id=1, variant_id=V1
=== SYSTEM PROMPT (V2) ===
You are an assistant that maps Polish medical service descriptions from insurance tender documents (SIWZ/OPZ) to internal service codes.
You receive:
- a short fragment of text describing a medical service in Polish,
- a list of possible service codes grouped by category and subcategory.

Your task is to return ONLY the codes that clearly match the described service.
If nothing fits, return an empty list.

=== USER PROMPT (V2) ===
Oto opis usługi medycznej wycięty z dokumentu (SIWZ / OPZ):

=== OPIS USŁUGI ===
Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.

Dodatkowe informacje o kontekście:
- Nagłówek bloku: Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.
- Kategoria bloku: other
- Czy profilakty

In [3]:
# Ile wariantów?
print("Variants in doc:", list(updated_doc.variant_items_by_id.keys()))

# Weźmy pierwszy wariant
first_variant_id = next(iter(updated_doc.variant_items_by_id.keys()))
items = updated_doc.variant_items_by_id[first_variant_id]

print(f"Liczba pozycji w wariancie {first_variant_id}:", len(items))

# Podejrzyjmy pierwsze np. 5 pozycji
for item in items[:5]:
    print("\n---")
    print(f"service_local_id: {item.service_local_id}")
    print(f"Tekst: {item.service_text}")
    print(f"Nagłówek: {item.block_heading_raw}")
    
    if item.predicted_services:
        codes = [c.code for c in item.predicted_services]
        labels = [c.label for c in item.predicted_services]
        print("Predicted codes:", codes)
        print("Labels:", labels)
    else:
        print("Predicted codes: BRAK (None/[])")
updated_doc

Variants in doc: ['V1']
Liczba pozycji w wariancie V1: 328

---
service_local_id: 1
Tekst: Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.
Nagłówek: Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.
Predicted codes: ['8989.089.00', '8989.089.03', '8989.089.08']
Labels: ['Porada lekarska, konsultacja, asysta', 'Porada, personel pomocniczy', 'Inna konsultacja']

---
service_local_id: 4.1
Tekst: lekarz podstawowej opieki medycznej/lekarz rodzinny
Nagłówek: Nielimitowany dostęp do konsultacji medycznych świadczonych przez następujących specjalistów:
Predicted codes: ['8989.089.00', '8989.089.02', '8989.089.07']
Labels: ['Porada lekarska, konsultacja, asysta', 'Porada lekarska, inne', 'Badanie podmiotowe']

---
service_local_id: 4.2
Tekst: internista
Nagłówek: Nielimitowany dostęp do konsultacji medycznych świadczonych przez następujących specjalistów:
Predicted codes: ['8989.089.00']
Labels: ['Porada lekarska, ko

ManualDocVariantItems(doc_id='OPZ_wybrane_strony', pdf_path='C:\\Programowanie\\SIWZ\\data\\examples\\OPZ_wybrane_strony.pdf', variant_items_by_id={'V1': [VariantItem(variant_id='V1', block_no='1', block_heading_raw='Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.', block_category='other', service_local_id='1', service_text='Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.', is_prophylaxis=False, is_occupational_medicine=False, is_telemedicine=False, is_pricing_only=False, source_segment_id='manual_p7_1_r2', page=7, extra=VariantItemExtra(from_manual=True, block_heading_raw='Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.'), expected_codes=None, predicted_services=[ServiceCandidate(code='8989.089.00', label='Porada lekarska, konsultacja, asysta', score=None, source='single_llm_all_codes_v2', rank=1), ServiceCandidate(code='8989.089.03', label='Porada, personel pomocnic

In [ ]:
print("=== RUN STATS ===")
print(stats.pretty())

=== RUN STATS ===
Processed items : 5
Elapsed time    : 117.99 s
Prompt tokens   : 656585
Completion tok. : 2494
Total tokens    : 659079
Est. cost (USD) : 0.1691



: 

In [ ]:
service_codes = load_service_codes_from_excel(excel_path)
client = GPTClient(model="gpt-5-mini", temperature=1)

strategy = SingleLLMMappingStrategyV3(
    client=client,
    service_codes=service_codes,
    blocks_above=5,
    blocks_below=5,
    segments_above=5,
    segments_below=5,
    max_examples_per_block=3,   # ile pozycji z każdego bloku pokazujemy
    debug=True,
    debug_log_prompts=True,
    debug_max_items=1,
)

c = 5  # ile pozycji przetworzyć na test

updated_doc, stats = run_strategy_on_json_path(
    json_path=json_path,
    strategy=strategy,
    output_path=output_path,
    max_items=c,
    show_progress=True,
)

print("=== RUN STATS V3 ===")
print(stats.pretty())


[DEBUG] SingleLLMMappingStrategyV3 prompt stats:
  service_text chars: 87
  codes in prompt   : 5143
  codes_block chars : 342520
  context chars     : 1821

[DEBUG PROMPT V3 #1] service_local_id=1, variant_id=V1
=== SYSTEM PROMPT (V3) ===
You are an assistant that maps Polish medical service descriptions from insurance tender documents (SIWZ/OPZ) to internal service codes.
You receive:
- a short fragment of text describing a medical service in Polish,
- a list of possible service codes grouped by category and subcategory,
- additional local context around this fragment: neighbouring blocks and segments.

Your task is to return ONLY the codes that clearly match the described service.
If nothing fits, return an empty list.

=== USER PROMPT (V3) ===
Oto opis usługi medycznej wycięty z dokumentu (SIWZ / OPZ):

=== OPIS USŁUGI ===
Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.

Dodatkowe informacje o segmencie:
- Nagłówek bloku: Całodobowa informacj

In [2]:
from siwz_mapper.eval.strategies import VariantWholeTextMappingStrategyV0

strategy_v0 = VariantWholeTextMappingStrategyV0(
    client=client,
    service_codes=service_codes,
    debug_log_prompts=True,   # żeby zobaczyć prompt i odpowiedź
)

with open(variant_txt_path, "r", encoding="utf-8") as f:
    variant_text = f.read()

result_v0 = strategy_v0.map_variant(variant_text)

print("Kody dla całego wariantu (V0):", result_v0.predicted_codes)



[DEBUG PROMPT V0]
=== SYSTEM PROMPT (V0) ===
You are an assistant that maps Polish medical insurance variant descriptions from tender documents (SIWZ/OPZ) to internal medical service codes.
You receive:
- the full text of one insurance variant (Polish),
- a list of all possible service codes grouped by category and subcategory.

Your task is to identify ALL codes that clearly appear in this variant (either explicitly or implicitly as described services).
If nothing fits, return an empty list.

=== USER PROMPT (V0) ===
Oto pełny tekst jednego wariantu z dokumentu (SIWZ / OPZ):

=== TEKST WARIANTU ===
Załącznik nr 2 A do OPZ zad. 1 - Minimalny wymagany zakres świadczeń w opiece medycznej – 
WARIANT 1 
 
Wymagany minimalny poziom świadczeń. 
 
1. Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.  
2. Rezerwacja wizyt: 
2.1.  za pośrednictwem infolinii dostępnej całą dobę, 
2.2.  przez Internet (wykonawca udostępni odpowiednią stronę internetową/formu

In [4]:
from siwz_mapper.eval.strategies import VariantWholeTextMappingStrategyV01
strategy_v01 = VariantWholeTextMappingStrategyV01(
    client=client,
    service_codes=service_codes,
    debug_log_prompts=True,   # podejrzenie promptu + odpowiedzi
)

with open(variant_txt_path, "r", encoding="utf-8") as f:
    variant_text = f.read()

resp_v01 = strategy_v01.map_variant(variant_text)

# np. zapisz do pliku jako czysty JSON:
output_path = r"C:\Programowanie\SIWZ\data\examples\variant1_mappings_v0_1.json"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(resp_v01.model_dump_json(indent=2))

print("Zapisano mapping V0.1 do:", output_path)

# Albo szybki podgląd:
for m in resp_v01.mappings[:5]:
    print("\n--- CHUNK ---")
    print(m.text_chunk)
    print("Kody:", m.codes)



[DEBUG PROMPT V0.1]
=== SYSTEM PROMPT (V0.1) ===
You are an assistant that maps Polish medical insurance variant descriptions from tender documents (SIWZ/OPZ) to internal medical service codes.
You receive:
- the full text of one insurance variant (Polish),
- a list of all possible service codes grouped by category and subcategory.

Your task is to:
1) split the variant text into meaningful chunks (e.g. bullet points, sentences or groups of sentences),
2) for each chunk, assign zero or more codes from the provided list.
If nothing fits for a given chunk, use an empty codes list.

=== USER PROMPT (V0.1) ===
Oto pełny tekst jednego wariantu z dokumentu (SIWZ / OPZ):

=== TEKST WARIANTU ===
Załącznik nr 2 A do OPZ zad. 1 - Minimalny wymagany zakres świadczeń w opiece medycznej – 
WARIANT 1 
 
Wymagany minimalny poziom świadczeń. 
 
1. Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.  
2. Rezerwacja wizyt: 
2.1.  za pośrednictwem infolinii dostępnej 

In [5]:
# Albo szybki podgląd:
for m in resp_v01.mappings[:100]:
    print("\n--- CHUNK ---")
    print(m.text_chunk)
    print("Kody:", m.codes)


--- CHUNK ---
1. Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.
Kody: ['8989.089.03']

--- CHUNK ---
2. Rezerwacja wizyt:
2.1. za pośrednictwem infolinii dostępnej całą dobę,
2.2. przez Internet (wykonawca udostępni odpowiednią stronę internetową/formularz),
Kody: []

--- CHUNK ---
3. Możliwość otrzymywania SMS z potwierdzeniem terminu świadczeń zdrowotnych.
Kody: []

--- CHUNK ---
4. Nielimitowany dostęp do konsultacji medycznych świadczonych przez następujących specjalistów: (lista specjalności: lekarz podstawowej opieki medycznej/lekarz rodzinny, internista, pediatra, chirurg ogólny, dermatolog, dermatolog wenerolog, ginekolog/ginekolog-położnik, otolaryngolog, neurolog, neurochirurg, okulista, pulmonolog, reumatolog, urolog, chirurg dziecięcy, urolog dziecięcy, otolaryngolog dziecięcy, neurolog dziecięcy)
Kody: ['8989.089.02']

--- CHUNK ---
5. Nielimitowane porady recepturowe (bez konsultacji lekarskiej).
Kody: ['8989.089.03']

--- CHUNK -

In [ ]:
from siwz_mapper.eval.strategies import VariantWholeTextMappingStrategyV03
service_codes = load_service_codes_from_excel(excel_path)
client = GPTClient(model="gpt-5-mini", temperature=1, timeout=1800.0)

strategy_v03 = VariantWholeTextMappingStrategyV03(
    client=client,
    service_codes=service_codes,
    num_code_batches=10,       # << ILE batchy chcesz
    debug_log_prompts=True,   # pokaże prompt dla pierwszego batcha
)

with open(variant_txt_path, "r", encoding="utf-8") as f:
    variant_text = f.read()

resp_v03 = strategy_v03.map_variant(variant_text)

# zapis do pliku
output_path = r"C:\Programowanie\SIWZ\data\examples\variant1_mappings_v0_3.json"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(resp_v03.model_dump_json(indent=2))

print("Zapisano mapping V0.3 do:", output_path)

# szybki podgląd kilku chunków
for m in resp_v03.mappings[:100]:
    print("\n--- CHUNK ---")
    print(m.text_chunk)
    print("Kody:", m.codes)



[DEBUG PROMPT V0.3 – first batch]
=== SYSTEM PROMPT (V0.3) ===
You are an assistant that maps Polish medical insurance variant descriptions from tender documents (SIWZ/OPZ) to internal medical service codes.
You receive:
- the full text of one insurance variant (Polish),
- a list of possible service codes grouped by category and subcategory.

Your task is to:
1) split the variant text into meaningful chunks (e.g. bullet points, sentences or groups of sentences),
2) for each chunk, assign zero or more codes from the provided list.
If nothing fits for a given chunk, use an empty codes list.

=== USER PROMPT (V0.3) ===
Oto pełny tekst jednego wariantu z dokumentu (SIWZ / OPZ):

=== TEKST WARIANTU ===
Załącznik nr 2 A do OPZ zad. 1 - Minimalny wymagany zakres świadczeń w opiece medycznej – 
WARIANT 1 
 
Wymagany minimalny poziom świadczeń. 
 
1. Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.  
2. Rezerwacja wizyt: 
2.1.  za pośrednictwem infolinii 

TypeError: BaseModel.model_dump_json() got an unexpected keyword argument 'ensure_ascii'

In [3]:
# zapis do pliku
output_path = r"C:\Programowanie\SIWZ\data\examples\variant1_mappings_v0_3.json"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(resp_v03.model_dump_json(indent=2))

print("Zapisano mapping V0.3 do:", output_path)

# szybki podgląd kilku chunków
for m in resp_v03.mappings[:100]:
    print("\n--- CHUNK ---")
    print(m.text_chunk)
    print("Kody:", m.codes)

Zapisano mapping V0.3 do: C:\Programowanie\SIWZ\data\examples\variant1_mappings_v0_3.json

--- CHUNK ---
1. Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.
Kody: []

--- CHUNK ---
2. Rezerwacja wizyt:
Kody: []

--- CHUNK ---
2.1.  za pośrednictwem infolinii dostępnej całą dobę,
Kody: []

--- CHUNK ---
2.2.  przez Internet (wykonawca udostępni odpowiednią stronę internetową/formularz),
Kody: []

--- CHUNK ---
3. Możliwość otrzymywania SMS z potwierdzeniem terminu świadczeń zdrowotnych.
Kody: []

--- CHUNK ---
4. Nielimitowany  dostęp  do  konsultacji  medycznych  świadczonych  przez  następujących specjalistów: (lista specjalistów: lekarz podstawowej opieki medycznej/lekarz rodzinny, internista, pediatra, chirurg ogólny, dermatolog, dermatolog wenerolog, ginekolog/ginekolog-położnik, otolaryngolog, neurolog, neurochirurg, okulista, pulmonolog, reumatolog, urolog, chirurg dziecięcy, urolog dziecięcy, otolaryngolog dziecięcy, neurolog dziecięcy)
Kod

In [2]:
from siwz_mapper.eval.strategies import MASVariantMappingStrategyV1, MASVariantMappingStrategyV11

service_codes = load_service_codes_from_excel(excel_path)
client = GPTClient(model="gpt-5-mini", temperature=1, timeout=1800.0)

with open(variant_txt_path, "r", encoding="utf-8") as f:
    variant_text = f.read()

# MASv1 – globalna lista kodów
mas_v1 = MASVariantMappingStrategyV1(
    client=client,
    service_codes=service_codes,
    max_chunk_chars=800,
    router_max_categories_per_chunk=None,  # albo po prostu nie podawać parametru
    router_examples_per_category=3,
    debug_log_prompts_router=True,
    debug_log_prompts_agents=False,
)


res_v1 = mas_v1.map_variant(variant_text)
print("Kody (MASv1):", res_v1.predicted_codes)

[MASv1] Chunk 1/1 (len=16533 chars)

[DEBUG CategoryRouterLLM – first chunk]
=== SYSTEM PROMPT ===
You are an assistant that routes Polish medical insurance text fragments (chunks) from tender documents (SIWZ/OPZ) to specialized category agents.
You receive:
- one text fragment in Polish,
- a list of all possible service categories with a few example services for each.

Your task is to choose all categories that are relevant for this fragment.
In practice, most fragments match 0–3 categories, but if more categories are clearly relevant, you may include all of them.

=== USER PROMPT ===
Oto fragment tekstu z dokumentu (SIWZ / OPZ):

=== FRAGMENT TEKSTU ===
Załącznik nr 2 A do OPZ zad. 1 - Minimalny wymagany zakres świadczeń w opiece medycznej – 
WARIANT 1 
 
Wymagany minimalny poziom świadczeń. 
 
1. Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.  
2. Rezerwacja wizyt: 
2.1.  za pośrednictwem infolinii dostępnej całą dobę, 
2.2.  przez Internet (

ServiceItemMappingResult(candidates=[ServiceCandidate(code='1212.012.02', label='Usunięcie ciała obcego z przedniej części gałki (bez magnesu)', score=None, source='mas_v1_union_codes_with_router', rank=1), ServiceCandidate(code='1212.212.29', label='Inne zabiegi diagnostyczne tęczówki, ciałka rzęskowego, twardówki i przedniej komory oka', score=None, source='mas_v1_union_codes_with_router', rank=2), ServiceCandidate(code='1818.118.11', label='Wziernikowanie ucha', score=None, source='mas_v1_union_codes_with_router', rank=3), ServiceCandidate(code='2121.021.01', label='Przednia tamponada nosa przy krwotoku', score=None, source='mas_v1_union_codes_with_router', rank=4), ServiceCandidate(code='2121.021.03', label='Przyżeganie przy krwotoku z nosa i tamponada', score=None, source='mas_v1_union_codes_with_router', rank=5), ServiceCandidate(code='2323.0423.0401', label='Znieczulenie miejscowe powierzchniowe', score=None, source='mas_v1_union_codes_with_router', rank=6), ServiceCandidate(cod

In [5]:

# MASv1.1 – chunk -> kody (JSON)
mas_v11 = MASVariantMappingStrategyV11(
    client=client,
    service_codes=service_codes,
    max_chunk_chars=800,
    router_max_categories_per_chunk=5,
    router_examples_per_category=3,
    debug_log_prompts_router=True,
    debug_log_prompts_agents=False,
)

resp_v11 = mas_v11.map_variant(variant_text)
print("Chunks:", len(resp_v11.mappings))
for m in resp_v11.mappings[:100]:
    print("\n--- CHUNK ---")
    print(m.text_chunk)
    print("Kody:", m.codes)

[MASv1.1] Chunk 1/1 (len=16533 chars)

[DEBUG CategoryRouterLLM – first chunk]
=== SYSTEM PROMPT ===
You are an assistant that routes Polish medical insurance text fragments (chunks) from tender documents (SIWZ/OPZ) to specialized category agents.
You receive:
- one text fragment in Polish,
- a list of all possible service categories with a few example services for each.

Your task is to choose ONLY those categories that are relevant for this fragment.
Return at most 5 categories.

=== USER PROMPT ===
Oto fragment tekstu z dokumentu (SIWZ / OPZ):

=== FRAGMENT TEKSTU ===
Załącznik nr 2 A do OPZ zad. 1 - Minimalny wymagany zakres świadczeń w opiece medycznej – 
WARIANT 1 
 
Wymagany minimalny poziom świadczeń. 
 
1. Całodobowa informacja o usługach i świadczeniach zdrowotnych dostępnych w ramach umowy.  
2. Rezerwacja wizyt: 
2.1.  za pośrednictwem infolinii dostępnej całą dobę, 
2.2.  przez Internet (wykonawca udostępni odpowiednią stronę internetową/formularz), 
. 
3. Możliwość otrzym

In [2]:
from siwz_mapper.eval.strategies import MASVariantMappingStrategyV2
from siwz_mapper.llm.gpt_client import GPTClient, print_llm_usage_summary

with open(variant_txt_path, "r", encoding="utf-8") as f:
    variant_text = f.read()

# V2 – union kodów
mas_v2 = MASVariantMappingStrategyV2(
    client=client,
    service_codes=service_codes,
    debug_log_prompts_planner=True,      # pokaż pełen prompt plannera
    debug_log_prompts_agents=True,       # włącz debug agentów
    debug_max_agents_to_log=3,           # tylko 3 pierwszych agentów pokaże prompty
)
res_v2 = mas_v2.map_variant(variant_text)
print("Kody (MASv2):", res_v2.predicted_codes)

# 1) Analiza kosztów
print_llm_usage_summary(client)

# 2) Raport wszystkich promptów + tokeny
client.print_debug_report(max_prompt_chars=400)



[DEBUG VariantPlannerRouterLLM – first call]
=== SYSTEM PROMPT ===
You are an assistant that plans and routes Polish medical insurance variant descriptions from tender documents (SIWZ/OPZ) to specialized category agents.
You receive:
- the full text of one insurance variant (Polish),
- a taxonomy of medical services: categories, subcategories and example services.

Your task is to:
1) Split the variant text into meaningful chunks (fragments). Each chunk should be a coherent unit:
   - one bullet point,
   - one numbered item,
   - or a small group of closely related sentences.
   Avoid extremely long chunks; if needed, break long sections into smaller ones (~1–5 sentences).
2) For each chunk, assign zero or more categories from the taxonomy, and for each category,    assign zero or more subcategories that are clearly relevant.
3) Do NOT invent new category or subcategory names – only use names that appear in the taxonomy.
4) Return the result as JSON matching the VariantPlan model.

=

In [3]:
# 1) Analiza kosztów
print_llm_usage_summary(client)

# 2) Raport wszystkich promptów + tokeny
client.print_debug_report(max_prompt_chars=10000000)


=== LLM USAGE SUMMARY ===
Model              : gpt-5-mini
Prompt tokens      : 111412
Completion tokens  : 37880
Total tokens       : 149292
Estimated cost (USD): 0.103613

=== LLM DEBUG REPORT ===
Total calls: 38

--- CALL #1 ---
Type: chat, model: gpt-5-mini
Prompt tokens: 70267, completion tokens: 15134

[SYSTEM PROMPT] (truncated)
You are an assistant that plans and routes Polish medical insurance variant descriptions from tender documents (SIWZ/OPZ) to specialized category agents.
You receive:
- the full text of one insurance variant (Polish),
- a taxonomy of medical services: categories, subcategories and example services.

Your task is to:
1) Split the variant text into meaningful chunks (fragments). Each chunk should be a coherent unit:
   - one bullet point,
   - one numbered item,
   - or a small group of closely related sentences.
   Avoid extremely long chunks; if needed, break long sections into smaller ones (~1–5 sentences).
2) For each chunk, assign zero or more categor

In [4]:
print("Kody (MASv2):", res_v2.predicted_codes)

Kody (MASv2): ['0606.106.11', '1212.012.02', '2020.820.82', '2121.021.01', '2121.021.03', '2929.129.19', '6262.162.19', '6767.167.13', '6767.167.14', '8585.185.11', '8686.086.04', '8686.086.05', '8686.286.23', '8686.286.27', '8686.286.28', '8686.586.52', '8686.586.53', '8686.586.54', '8686.586.55', '8787.287.21', '8787.287.22', '8787.287.23', '8787.287.24', '8787.287.29', '8787.487.43', '8787.487.44', '8787.487.49', '8888.788.71', '8888.788.75', '8888.788.76', '8888.788.78', '8888.788.79', '8989.089.00', '8989.089.03', '8989.089.04', '8989.089.07', '8989.589.52', '9191.691.62', '9595.095.03', '9595.095.05', '9595.295.26', '9797.297.29', '9898.198.11', '9898.198.12', '9898.198.13', '9999.299.29', '9999.599.52', 'AAAA01', 'AAAA07', 'AAAA09', 'AAAA11', 'AAAA15', 'AAAA17', 'AAAA21', 'AAAA25', 'AACC02', 'AACC19', 'AACC30', 'AACC32', 'AACC53', 'AACC55', 'AACC59', 'AACC66', 'AACC69', 'AAEE65', 'AAGG11', 'AAGG15', 'AAGG19', 'AAGG21', 'AAGG25', 'AAGG53', 'AAII09', 'AAII17', 'AAII19', 'AAII25', 

In [ ]:
import os
# 🔴 Najprościej (ale NIE wrzucaj tego do repo!)
os.environ["OPENAI_API_KEY"] = ""
from siwz_mapper.eval.codebook import load_service_codes_from_excel
from siwz_mapper.eval.strategies import SingleLLMMappingStrategyV3
from siwz_mapper.eval.runner import run_strategy_on_json_path
from siwz_mapper.llm.gpt_client import GPTClient
json_path = r"C:\Programowanie\SIWZ\data\examples\OPZ_manual_variant_items.json"
excel_path = r"C:\Programowanie\SIWZ\data\examples\kody_mapowanie.xlsx"
output_path = r"C:\Programowanie\SIWZ\data\examples\OPZ_wybrane_strony_with_predictions_v2.json"
variant_txt_path = r"C:\Programowanie\SIWZ\data\examples\wariantV1.txt"

service_codes = load_service_codes_from_excel(excel_path)
client = GPTClient(model="gpt-5-mini", temperature=1,  timeout=1800.0)

In [27]:
import pickle
from pathlib import Path

CACHE_PATH = Path("opz_cache.pkl")

to_save = {}

# tylko jeśli istnieją – niektóre mogą nie być zdefiniowane
for name in [
    "last_blocks",
    "last_segments",
    "last_classifications",
    "last_updated_segments",
    "last_variant_groups",
]:
    if name in globals():
        to_save[name] = globals()[name]

print("Zapisuję pola:", list(to_save.keys()))

with open(CACHE_PATH, "wb") as f:
    pickle.dump(to_save, f)

print("Zapisano cache do:", CACHE_PATH)


Zapisuję pola: ['last_blocks', 'last_segments', 'last_classifications', 'last_updated_segments', 'last_variant_groups']
Zapisano cache do: opz_cache.pkl


In [4]:
import pickle
from pathlib import Path

CACHE_PATH = Path("opz_cache.pkl")

with open(CACHE_PATH, "rb") as f:
    loaded = pickle.load(f)

for name, value in loaded.items():
    globals()[name] = value

print("Załadowano z cache:", list(loaded.keys()))


Załadowano z cache: ['last_blocks', 'last_segments', 'last_classifications', 'last_updated_segments', 'last_variant_groups']
